First step is to read, and preprocess the data.

In [ ]:
%tensorflow_version 1.x
import tensorflow

TensorFlow 1.x selected.


In [ ]:
#imports
import os
import time
from google.colab import drive
import numpy as np
import matplotlib as plt
from matplotlib import pyplot
import pandas as pd

# Import keras modules
from keras.models import Sequential
from keras.layers import Dense

import numpy as np
import keras
from keras import models, layers, Input, regularizers
from keras.utils import np_utils
from keras.models import Sequential, Model
import matplotlib.pyplot as plt
from keras.layers import Dense
import random
random.seed(42)

from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.optimizers import SGD
from keras import backend as K
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

Using TensorFlow backend.


In [ ]:
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/MRP/Covid-target/self/')

In [ ]:
#Toggles
self_sup = True
incep = True
aug = True
saves = False
big_arch = 0
self_frac = "50"
file_name = "50Exp2_full"

In [ ]:
start_time = time.clock()


In [ ]:
#I will try Inception, so change H,W
if incep == True:
    H=299
    W=299
else:
    H=224
    W=224



In [ ]:
#End of preprocessing

In [ ]:
if incep == True:
    base_model = keras.applications.InceptionV3(
        include_top = False,
        weights = "imagenet")
else:
    base_model = keras.applications.DenseNet169(
        include_top = False,
        weights = "imagenet")    

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


87916544/87910968 [==============================] - 2s 0us/step


Next step is to load the pre-trained model, add our own dense layer on top, and train the dense layer.

In [ ]:
if self_sup == True:
    cp = ModelCheckpoint(filepath= file_name + '_run1.hdf5', verbose=1, monitor='val_loss', save_best_only=True)

In [ ]:
if aug == True:
    train_datagen = keras.preprocessing.image.ImageDataGenerator(
                                                dtype='float64')

    #on test, we do not augment, merely preprocess
    test_datagen = keras.preprocessing.image.ImageDataGenerator(
                                                dtype='float64')
    
    train_generator = train_datagen.flow_from_directory(
        self_frac + 'train',
        target_size = (W,H),
        #batch_size = 32,
        class_mode='categorical'
    )

    test_generator = test_datagen.flow_from_directory(
        self_frac + 'test',
        target_size = (W,H),
        #batch_size = 16,
        class_mode = 'categorical'
    )

Found 544 images belonging to 2 classes.
Found 204 images belonging to 2 classes.


In [ ]:
#no fine tuning

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

if big_arch == 1:
    x = layers.BatchNormalization()(x)
    x = layers.Dense(1024, activation = 'relu', kernel_regularizer = regularizers.l2(0.01),)(x) 
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(1024, activation = 'relu', kernel_regularizer = regularizers.l2(0.01),)(x) 
    x = layers.Dropout(0.5)(x)    

else:
    x = layers.BatchNormalization()(x)
    x = layers.Dense(1024, activation = 'relu', kernel_regularizer = regularizers.l2(0.01),)(x) 
    x = layers.Dropout(0.5)(x)

predictions = Dense(2, activation='sigmoid')(x)


model = Model(inputs=base_model.input, outputs=predictions)


if self_sup == True:
    
    for layer in base_model.layers:
        layer.trainable = False

    
    model.compile(optimizer=keras.optimizers.RMSprop(lr=0.0001), loss='binary_crossentropy', metrics=["accuracy"])

    
    if aug == True:
        hist = model.fit_generator(train_generator, epochs=30, validation_data=test_generator, callbacks=[cp], verbose=1)
    else:
        hist1 = model.fit(x=train_img,y = y_train, epochs=30, validation_data=(test_img, y_test), callbacks=[cp], verbose=1)



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/30
17/17 [==============================] - 185s 11s/step - loss: 14.2537 - accuracy: 0.4816 - val_loss: 16.6655 - val_accuracy: 0.4951

Epoch 00001: val_loss improved from inf to 16.66549, saving model to 50Exp2_full_run1.hdf5
Epoch 2/30
17/17 [==============================] - 2s 120ms/step - loss: 13.7181 - accuracy: 0.5165 - val_loss: 17.2690 - val_accuracy: 0.5123

Epoch 00002: val_loss did not improve from 16.66549
Epoch 3/30
17/17 [==============================] - 3s 155ms/step - loss: 13.2877 - accuracy: 0.5368 - val_loss: 17.2765 - val_accuracy: 0.5000

Epoch 00003: val_loss did not improve from 16.66549
Epoch 4/30
17/17 [==============================] - 3s 158ms/step - loss: 12.8397 - accuracy: 0.5754 - val_loss: 24.4769 - val_accuracy: 0.4902

Epoch 00004: val_loss did not improve from 16.66549
Epoch 5/30
17/17 [==============================] - 3s 161ms/step - loss: 12.4

In [ ]:
if self_sup == True:
    cp2 = ModelCheckpoint(filepath= file_name + '_run2.hdf5', verbose=1, monitor='val_loss', save_best_only=True)

In [ ]:
if aug == True:
    train_generator = train_datagen.flow_from_directory(
        self_frac + 'train',
        target_size = (W,H),
        batch_size = 4,
        class_mode='categorical'
    )

    test_generator = test_datagen.flow_from_directory(
        self_frac + 'test',
        target_size = (W,H),
        #batch_size = 16,
        class_mode = 'categorical'
    )

Found 544 images belonging to 2 classes.
Found 204 images belonging to 2 classes.


In [ ]:
if self_sup == True:
    #Fine tuning

    from keras.models import load_model
    model = load_model(file_name + '_run1.hdf5')


    for layer in model.layers:
      layer.trainable = True

    
    from keras.optimizers import SGD
    model.compile(optimizer=SGD(lr=0.00001, momentum=0.9), loss='binary_crossentropy', metrics=["accuracy"])

    
    if aug == True:
        hist = model.fit_generator(train_generator, epochs=30, validation_data=test_generator, callbacks=[cp2], verbose=1)
    else:
        hist1 = model.fit(x=train_img,y = y_train, epochs=30, validation_data=(test_img, y_test), callbacks=[cp2], verbose=1)

    

Epoch 1/30
136/136 [==============================] - 22s 161ms/step - loss: 7.1254 - accuracy: 0.5680 - val_loss: 7.4362 - val_accuracy: 0.4828

Epoch 00001: val_loss improved from inf to 7.43625, saving model to 50Exp2_full_run2.hdf5
Epoch 2/30
136/136 [==============================] - 12s 86ms/step - loss: 7.1418 - accuracy: 0.5607 - val_loss: 7.0307 - val_accuracy: 0.5196

Epoch 00002: val_loss improved from 7.43625 to 7.03070, saving model to 50Exp2_full_run2.hdf5
Epoch 3/30
136/136 [==============================] - 12s 87ms/step - loss: 7.1468 - accuracy: 0.5662 - val_loss: 7.1897 - val_accuracy: 0.5245

Epoch 00003: val_loss did not improve from 7.03070
Epoch 4/30
136/136 [==============================] - 12s 87ms/step - loss: 7.1085 - accuracy: 0.5616 - val_loss: 7.0959 - val_accuracy: 0.5294

Epoch 00004: val_loss did not improve from 7.03070
Epoch 5/30
136/136 [==============================] - 12s 87ms/step - loss: 7.1002 - accuracy: 0.5910 - val_loss: 7.1123 - val_accura

In [ ]:
if self_sup == True:
    #learning rate decay
    
    lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5)
    cp3 = ModelCheckpoint(filepath=file_name + '_run3.hdf5', verbose=1, monitor='val_loss', save_best_only=True)

In [ ]:
if self_sup == True:
    model = load_model(file_name + '_run2.hdf5')
    model.compile(optimizer=SGD(lr=0.000001), loss='binary_crossentropy', metrics=["accuracy"])

    
    if aug == True:
        hist = model.fit_generator(train_generator, epochs=30, validation_data=test_generator, callbacks=[cp3], verbose=1)
    else:
        hist1 = model.fit(x=train_img,y = y_train, epochs=30, validation_data=(test_img, y_test), callbacks=[cp3], verbose=1)

  

Epoch 1/30
136/136 [==============================] - 22s 165ms/step - loss: 6.9482 - accuracy: 0.6765 - val_loss: 7.2180 - val_accuracy: 0.5172

Epoch 00001: val_loss improved from inf to 7.21804, saving model to 50Exp2_full_run3.hdf5
Epoch 2/30
136/136 [==============================] - 12s 87ms/step - loss: 6.9764 - accuracy: 0.6507 - val_loss: 7.0938 - val_accuracy: 0.5025

Epoch 00002: val_loss improved from 7.21804 to 7.09384, saving model to 50Exp2_full_run3.hdf5
Epoch 3/30
136/136 [==============================] - 12s 88ms/step - loss: 6.9536 - accuracy: 0.6654 - val_loss: 7.0295 - val_accuracy: 0.5074

Epoch 00003: val_loss improved from 7.09384 to 7.02947, saving model to 50Exp2_full_run3.hdf5
Epoch 4/30
136/136 [==============================] - 12s 88ms/step - loss: 6.9585 - accuracy: 0.6719 - val_loss: 7.3419 - val_accuracy: 0.5172

Epoch 00004: val_loss did not improve from 7.02947
Epoch 5/30
136/136 [==============================] - 12s 88ms/step - loss: 6.9484 - accur

In [ ]:
#Try on benign vs malignant
os.chdir('/content/drive/My Drive/MRP/Covid-target/organized/')

In [ ]:
org_base = '/content/drive/My Drive/MRP/Covid-target/self/'

In [ ]:
lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5)
cp4 = ModelCheckpoint(filepath=file_name + '_run4.hdf5', verbose=1, monitor='val_accuracy', save_best_only=True)

In [ ]:
if aug == True:
    train_generator = train_datagen.flow_from_directory(
        'big_train',
        target_size = (W,H),
        #batch_size = 32,
        class_mode='categorical'
    )

    test_generator = test_datagen.flow_from_directory(
        'test',
        target_size = (W,H),
        #batch_size = 16,
        class_mode = 'categorical'
    )


Found 543 images belonging to 2 classes.
Found 203 images belonging to 2 classes.


In [ ]:
#benign vs malignant
if self_sup == True:
    model = load_model(org_base + file_name + '_run3.hdf5')


for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=keras.optimizers.RMSprop(lr=0.0001), loss='binary_crossentropy', metrics=["accuracy"])


if aug == True:
    hist = model.fit_generator(train_generator, epochs=30, validation_data=test_generator, callbacks=[cp4], verbose=1)
else:
    hist1 = model.fit(x=train_img,y = y_train, epochs=30, validation_data=(test_img, y_test), callbacks=[cp4], verbose=1)


Epoch 1/30
17/17 [==============================] - 152s 9s/step - loss: 6.7916 - accuracy: 0.7192 - val_loss: 6.6142 - val_accuracy: 0.8300

Epoch 00001: val_accuracy improved from -inf to 0.83005, saving model to 50Exp2_full_run4.hdf5
Epoch 2/30
17/17 [==============================] - 5s 321ms/step - loss: 5.9794 - accuracy: 0.9834 - val_loss: 6.6967 - val_accuracy: 0.8325

Epoch 00002: val_accuracy improved from 0.83005 to 0.83251, saving model to 50Exp2_full_run4.hdf5
Epoch 3/30
17/17 [==============================] - 5s 322ms/step - loss: 5.5186 - accuracy: 0.9963 - val_loss: 6.4887 - val_accuracy: 0.8202

Epoch 00003: val_accuracy did not improve from 0.83251
Epoch 4/30
17/17 [==============================] - 5s 322ms/step - loss: 4.9262 - accuracy: 1.0000 - val_loss: 5.9685 - val_accuracy: 0.7882

Epoch 00004: val_accuracy did not improve from 0.83251
Epoch 5/30
17/17 [==============================] - 5s 320ms/step - loss: 4.3542 - accuracy: 0.9843 - val_loss: 4.3766 - val_a

In [ ]:
if aug == True:
    train_generator = train_datagen.flow_from_directory(
        'big_train',     # changed to big_train. if 425 is fisrst number below, this needs to be re run.
        target_size = (W,H),
        batch_size = 4,
        class_mode='categorical'
    )

    test_generator = test_datagen.flow_from_directory(
        'test',
        target_size = (W,H),
        #batch_size = 16,
        class_mode = 'categorical'
    )

Found 543 images belonging to 2 classes.
Found 203 images belonging to 2 classes.


In [ ]:
cp5 = ModelCheckpoint(filepath=file_name + '_run5.hdf5', verbose=1, monitor='val_accuracy', save_best_only=True)

In [ ]:
#model = load_model(file_name + '_run4.hdf5')
for layer in model.layers:
   layer.trainable = True


from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.00001, momentum=0.9), loss='binary_crossentropy', metrics=["accuracy"])


if aug == True:
    hist = model.fit_generator(train_generator, epochs=30, validation_data=test_generator, callbacks=[cp5], verbose=1)
else:
    hist1 = model.fit(x=train_img,y = y_train, epochs=30, validation_data=(test_img, y_test), verbose=1)


Epoch 1/30
136/136 [==============================] - 24s 180ms/step - loss: 0.5689 - accuracy: 0.8987 - val_loss: 0.5095 - val_accuracy: 0.8473

Epoch 00001: val_accuracy improved from -inf to 0.84729, saving model to 50Exp2_full_run5.hdf5
Epoch 2/30
136/136 [==============================] - 12s 91ms/step - loss: 0.4172 - accuracy: 0.9227 - val_loss: 0.3277 - val_accuracy: 0.8621

Epoch 00002: val_accuracy improved from 0.84729 to 0.86207, saving model to 50Exp2_full_run5.hdf5
Epoch 3/30
136/136 [==============================] - 12s 90ms/step - loss: 0.4168 - accuracy: 0.9190 - val_loss: 0.5545 - val_accuracy: 0.8571

Epoch 00003: val_accuracy did not improve from 0.86207
Epoch 4/30
136/136 [==============================] - 12s 88ms/step - loss: 0.4224 - accuracy: 0.9125 - val_loss: 0.7651 - val_accuracy: 0.8547

Epoch 00004: val_accuracy did not improve from 0.86207
Epoch 5/30
136/136 [==============================] - 12s 88ms/step - loss: 0.4940 - accuracy: 0.8959 - val_loss: 0.

In [ ]:
total_time = time.clock() - start_time
print(total_time)

2719.6700220000002


In [ ]:
model = load_model(file_name + '_run5.hdf5')

In [ ]:
generator = test_datagen.flow_from_directory(
        'test',
        target_size=(W, H),
        batch_size=1,
        class_mode=None,  # only data, no labels
        shuffle=False)  # keep data in same order as labels

probabilities = model.predict_generator(generator, 203)

Found 203 images belonging to 2 classes.


In [ ]:
from sklearn.metrics import confusion_matrix, f1_score, roc_auc_score, accuracy_score

y_true = np.array([0] * 98 + [1] * 105)
y_pred = np.argmax(probabilities, axis = 1)

confusion_matrix(y_true, y_pred.round())

array([[82, 16],
       [12, 93]])

In [ ]:
print(f1_score(y_true, y_pred))
print()
print(roc_auc_score(y_true, y_pred))
print()
print(accuracy_score(y_true, y_pred))

0.8691588785046729

0.8612244897959185

0.8620689655172413
